<a href="https://colab.research.google.com/github/chyj0805/Project_2/blob/main/Fetures/Data_fetches_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-dotenv

In [6]:
from dotenv import load_dotenv
import requests
import pandas as pd
import os

class StockData:
    def __init__(self):
        load_dotenv()
        self.api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
        if self.api_key is None:
            raise ValueError("Alpha Vantage API key is not set in the environment.")

    def get_stock_historical_data_with_MACD(self, stock_symbol, interval="daily", series_type="close"):
        base_url = "https://www.alphavantage.co/query"

        stock_params = {
            "function": "TIME_SERIES_DAILY",
            "symbol": stock_symbol,
            "apikey": self.api_key,
            "outputsize": "full",
            "datatype": "json"
        }

        response = requests.get(base_url, params=stock_params)
        stock_data = response.json()

        if "Time Series (Daily)" not in stock_data:
            print("Error fetching stock data. Check your API key and stock symbol.")
            return None

        time_series = stock_data["Time Series (Daily)"]
        stock_list = []

        for date, details in time_series.items():
            row = {
                "Date": date,
                "Open": details["1. open"],
                "High": details["2. high"],
                "Low": details["3. low"],
                "Close": details["4. close"],
                "Volume": details["5. volume"]
            }
            stock_list.append(row)

        stock_df = pd.DataFrame(stock_list)
        stock_df["Date"] = pd.to_datetime(stock_df["Date"])
        stock_df = stock_df.sort_values(by="Date")

        macd_params = {
            "function": "MACD",
            "symbol": stock_symbol,
            "interval": interval,
            "series_type": series_type,
            "apikey": self.api_key,
            "datatype": "json"
        }

        response = requests.get(base_url, params=macd_params)
        macd_data = response.json()

        if "Technical Analysis: MACD" not in macd_data:
            print("Error fetching MACD data. Check your API key and stock symbol.")
            return None

        macd_series = macd_data["Technical Analysis: MACD"]
        macd_list = []

        for date, details in macd_series.items():
            row = {
                "Date": date,
                "MACD_Signal": details["MACD_Signal"],
                "MACD_Hist": details["MACD_Hist"],
                "MACD": details["MACD"],
            }
            macd_list.append(row)

        macd_df = pd.DataFrame(macd_list)
        macd_df["Date"] = pd.to_datetime(macd_df["Date"])
        macd_df = macd_df.sort_values(by="Date")

        merged_df = pd.merge(stock_df, macd_df, on="Date", how="inner")

        return merged_df

    def get_combined_data(self, stock_symbol, interval="daily", series_type="close"):
    # Load the environment variable file and get API key
      load_dotenv()
      api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

      if api_key is None:
          raise ValueError("Alpha Vantage API key is not set in the environment.")

      # Fetch Stock and MACD Data
      stock_macd_df = self.get_stock_historical_data_with_MACD(stock_symbol, interval, series_type)

      # Fetch Federal Funds Rate Data
      rate_url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval={interval}&apikey={api_key}'
      rate_data = requests.get(rate_url).json()

      if 'data' not in rate_data:
          print("Error fetching FEDERAL_FUNDS_RATE data. Check your API key.")
          return None

      fed_dates = [entry['date'] for entry in rate_data['data']]
      fed_rates = [float(entry['value']) for entry in rate_data['data']]
      fed_df = pd.DataFrame({'Federal Funds Rate': fed_rates}, index=fed_dates)
      fed_df.index = pd.to_datetime(fed_df.index)
      fed_df.sort_index(ascending=True, inplace=True)

      # Fetch Unemployment Data
      job_url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={api_key}'
      job_data = requests.get(job_url).json()

      if 'data' not in job_data:
          print("Error fetching UNEMPLOYMENT data. Check your API key.")
          return None

      job_dates = [entry['date'] for entry in job_data['data']]
      job_rates = [float(entry['value']) for entry in job_data['data']]
      job_df = pd.DataFrame({'Unemployment Rate': job_rates}, index=job_dates)
      job_df.index = pd.to_datetime(job_df.index)
      job_df = job_df.resample('D').ffill()

      # Fetch EMA 20 Data
      ema_url = f'https://www.alphavantage.co/query?function=EMA&symbol={stock_symbol}&interval={interval}&time_period=20&series_type={series_type}&apikey={api_key}'
      ema_data = requests.get(ema_url).json()
      ema_df = pd.DataFrame(ema_data['Technical Analysis: EMA']).T.reset_index()
      ema_df.columns = ['Date', 'EMA20']
      ema_df['Date'] = pd.to_datetime(ema_df['Date'])
      ema_df.sort_index(ascending=True, inplace=True)

      # Fetch CPI Data
      cpi_url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={api_key}'
      cpi_r = requests.get(cpi_url)
      cpi_data = cpi_r.json()['data']
      cpi_dates = [entry['date'] for entry in cpi_data]
      cpi_values = [float(entry['value']) for entry in cpi_data]
      cpi_df = pd.DataFrame({'CPI': cpi_values}, index=cpi_dates)
      cpi_df.index = pd.to_datetime(cpi_df.index)
      cpi_df = cpi_df.resample('D').ffill()

      # Merge all DataFrames
      stock_macd_df['Date'] = pd.to_datetime(stock_macd_df['Date'])
      combined_df = pd.merge(stock_macd_df, fed_df, left_on='Date', right_index=True, how='left')
      combined_df = pd.merge(combined_df, job_df, left_on='Date', right_index=True, how='left')
      combined_df = pd.merge(combined_df, ema_df, on='Date', how='left')
      combined_df = pd.merge(combined_df, cpi_df, left_on='Date', right_index=True, how='left')

      # Drop rows with NaN values
      combined_df.dropna(inplace=True)
      combined_df.fillna(method='ffill', inplace=True)
      return combined_df

# Usage
stock_data_instance = StockData()
combined_data = stock_data_instance.get_combined_data("AAPL")


In [7]:
combined_data = StockData()

In [8]:
IYE_df = combined_data.get_combined_data("IYE")
IYE_df

,Date,Open,High,Low,Close,Volume,MACD_Signal,MACD_Hist,MACD,Federal Funds Rate,Unemployment Rate,EMA20,CPI
0,2000-08-03,49.1900,49.1900,48.5000,48.5300,1500,-0.2024,0.0831,-0.1194,6.45,4.1,9.5533,172.800
1,2000-08-04,47.7500,48.5300,47.7500,48.5300,2300,-0.1809,0.0859,-0.0950,6.44,4.1,9.5671,172.800
2,2000-08-07,48.8800,49.2800,48.8800,49.1300,1000,-0.1578,0.0925,-0.0652,6.46,4.1,9.5910,172.800
3,2000-08-08,49.0300,49.3100,49.0300,49.3100,400,-0.1339,0.0956,-0.0383,6.44,4.1,9.6161,172.800
4,2000-08-09,50.0600,50.4100,50.0000,50.2200,7200,-0.1076,0.1053,-0.0023,6.48,4.1,9.6561,172.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5758,2023-06-26,40.8300,41.7499,40.8300,41.5100,343981,-0.0949,-0.0662,-0.1611,5.07,3.6,41.7665,305.109
5759,2023-06-27,41.4500,41.8000,41.2600,41.6600,280854,-0.1061,-0.0445,-0.1505,5.07,3.6,41.7564,305.109
5760,2023-06-28,41.6400,42.1000,41.2800,42.0600,502234,-0.1066,-0.0020,-0.1086,5.07,3.6,41.7853,305.109
5761,2023-06-29,42.2100,42.5400,42.0200,42.5400,1057954,-0.0925,0.0563,-0.0362,5.07,3.6,41.8572,305.109
